# Importa dependências

In [16]:
import json
import os
import glob
import re
import os.path as osp
import pathlib
import tarfile
import bz2
from zipfile import ZipFile
from pyunpack import Archive
import pandas as pd

# Define Origem (SRC) e Destino (DST)

In [17]:
src_path = "../data/documentos_json"
dst_path = "../data/documentos_json_by_doc"

In [41]:
base_path = "../data/"
anos = [2017,2018,2019,2020]

anos_list = []
for ano in anos:
    filename = f'{base_path}351-licitacoes-itamarati/licitacoes_itamarati_{str(ano)}/data/files/file_description.jsonl'
    df = pd.read_json(filename, lines=True)
    df['ano'] = str(ano)
    anos_list.append(df)
df_description = pd.concat(anos_list)
df_description.reset_index(inplace=True, drop=True)
df_description
df_description["file_id"] = df_description.file_name.apply(lambda v: str(v).rstrip(".pdf"))
mapper = df_description.loc[:, ["file_id", "ano"]].set_index("file_id").to_dict("index")

In [49]:
dirs = {
    "sao-bento-abade": "381-licitacoes-sao-bento-abade",
    "olaria": "302-licitacoes-olaria",
    "coqueiral": "289-licitacoes-coqueiral",
    "cristais": "290-licitacoes-cristais",
    "pirapetinga": "288_licitacoes-pirapetinga",
    "passa-vinte": "304-licitacoes-passa-vinte",
    "arantina": "306-licitacoes-arantina",
    "ijaci": "353-licitacoes-ijaci",
    "itamarati": "351-licitacoes-itamarati",
    "ribeirao-vermelho": "385-licitacoes-ribeirao-vermelho",
}

# Converte JSON/Cidade para JSON/doc

In [52]:
cities = glob.glob(src_path + "/*.json")
for city in cities:
    city_name = city.split("/")[-1].split(".")[0]
    city_path = f"../data/{dirs[city_name]}"
    
    # Divide em JSON em JSON menores, ie, um JSON por doc ao invés de um JSON por cidade
    with open(city) as f:
        i = 0
        for doc in f:
            i += 1
            doc = json.loads(doc)
            if doc["status"] == "SUCCESS":
                try:
                    file_id = doc["file_id"]
                    base_path = city_path
                    if city_name == "itamarati":
                        base_path += "itamarati_" + mapper[file_id]["ano"]
                    
                    base_path += "/data/files_json/"
                    
                    if not os.path.isdir(base_path):
                        os.mkdir(base_path)
                    
                    with open(f"{base_path}/{file_id}.json", "wt", encoding="utf-8") as dst_f:
                       json.dump(doc, dst_f)
                except:
                    pass
        print(city, i)


../data/documentos_json/sao-bento-abade.json 232
../data/documentos_json/olaria.json 42
../data/documentos_json/coqueiral.json 1528
../data/documentos_json/cristais.json 1736
../data/documentos_json/pirapetinga.json 1007
../data/documentos_json/passa-vinte.json 395
../data/documentos_json/arantina.json 983
../data/documentos_json/ijaci.json 451
../data/documentos_json/itamarati.json 1110
../data/documentos_json/ribeirao-vermelho.json 684


In [32]:
mapper["72b0eb3a7dba3bc83522d8da5114f36d"]

KeyError: '72b0eb3a7dba3bc83522d8da5114f36d'